# Исследование надежности заемщиков.

## Описание проекта
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Задачи:**
- Есть ли зависимость между количеством детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

**План работы:**
- Ознакомиться с общей информацией о данных.
- Провести их предобработку, а имеено:
  - заполнить пропущенные значения;
  - проверить наличие артефактов;
  - привести к типам данных, наиболее подходящим для работы;
  - удалить дубликаты;
  - выделить отдельные датафреймы для удобства рабтоты;
  - добавити новые столбцы с категоризицией.

**Описание данных**
- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита


## Обзор данных

In [17]:
# импорт библиотек
import pandas as pd

In [18]:
# загружаю данные
df = pd.read_csv('data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [19]:
# проверяю пропуски
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Выводы**

В столбцах `days_employed` и `total_income` есть пропущенные значения, в столбце `days_employed` некоторые значения отрицательные, что на первый взгляд невозможно по смыслу, также данные в этом столбце, как и в `total_income` вещественные, возможно для обработки будет удобнее использовать целочисленные данные. В столбце `education` данные нужно будет привести к единому регистру, чтобы упростить процесс категоризации, а в `purpose` проработать смысловые дубликаты. Количество пропущенных значений в столбцах `days_employed` и `total_income` одинаковое и составляет 10.1% от всех данных клиентов, это существенный объем, поэтому для анализа данных потребуется заполнение этих пропусков.

## Предобработка данных

### Заполнение пропусков

In [20]:
# проверяю пропуски в total_income
df[df['total_income'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Пропуски в total_income совпадают с days_employed, из чего можно сделать вывод, что значения пропущены, так как клиент не работает в этот момент. Так как нет возможности уточнить данные, а доля их существенная, потребуется подобрать оптимальный способ заполнения пропусков.

In [21]:
df['total_income'].describe()

count    1.935100e+04
mean     1.674223e+05
std      1.029716e+05
min      2.066726e+04
25%      1.030532e+05
50%      1.450179e+05
75%      2.034351e+05
max      2.265604e+06
Name: total_income, dtype: float64

Минимальное и максимальное значения в уровне дохода различается колоссально, поэтому пропуску в `total_income` целесообразнее заполнить медианным значением. В `days_employed` перед заполнением пропусков стоит проверить качество указанных данных. 

In [22]:
# заполняю пропуски
total_income_median = df['total_income'].median()
df['total_income'] = df['total_income'].fillna(total_income_median)

### Проверка данных на аномалии.

In [23]:
# проверяю children
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

У 47 клиентов количество детей отрицательное, вероятно это ошибка в написании, поэтому эти значения можно заменить на модульные. У 76 клиентов количество детей равно 20, похоже, что и в этих данных также ошибка, это могло произойти механически, когда клиент дописал лишний ноль, указывая значение 2, так как требуется оценить, как количество детей влияет на платежеспособность, то данные лучше не удалять, а заменить значения 20 на 2.

In [24]:
# избавляюсь от аномалий
df['children'] = abs(df['children'])
df['children'] = df['children'].replace(20, 2)

In [25]:
# проверяю dob_years
df['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

У 101 клиента в dob_years указанно 0, это явная ошибка, но возраст клиентов не влияет на выводы исследования, поэтому эти данные можно оставить без изменений.

In [26]:
# проверяю аномалии в стаже
df[df['days_employed'] >= df['dob_years'] * 365 - 18 * 365].sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12542,0,352818.624172,57,среднее,1,женат / замужем,0,F,пенсионер,0,42198.608938,автомобиль
18293,0,393990.553333,54,среднее,1,женат / замужем,0,F,пенсионер,0,99868.054154,на покупку подержанного автомобиля
5637,0,373778.073012,55,среднее,1,Не женат / не замужем,4,F,пенсионер,0,54430.602216,дополнительное образование
19291,0,366632.069693,56,среднее,1,гражданский брак,1,F,пенсионер,1,91719.837456,на проведение свадьбы
18103,1,349545.902003,67,среднее,1,женат / замужем,0,F,пенсионер,0,93093.261296,покупка жилья для сдачи


Можно предположить, что некоторые клиенты начали работать до совершеннолетия, поэтому их трудовой стаж превышает значение "возраст минус 18", но это не объясняет, почему трудовой стаж записан отрицательным значением и почему встречаются значения ощутимо превышающие самый оптимистичный срок продолжительности жизни. Заменим отрицательные значения положительными и вычислим медианное значение days_employed и заполним им пропуски. Так как продолжительность трудового стажа не влияет на ход исследования, можно оставить аномальные значения без изменений и не исключать из выборки.

In [27]:
df['days_employed'] = abs(df['days_employed'])
days_employed_median = df['days_employed'].median()
df['days_employed'] = df['days_employed'].fillna(days_employed_median)

In [28]:
# проверяю аномалии в gender
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

В столбце gender обнаружено 1 аномальное обозначение пола, роли это не сыграет, но для порядка можно удалить эту строку.

In [29]:
df = df[df['gender'] != 'XNA']

### Изменение типов данных.

In [30]:
df['total_income'] = df['total_income'].astype('int')

### Удаление дубликатов.

In [31]:
# проверяю неявные дубликаты в education
df['education'] = df['education'].str.lower()
df.duplicated().sum()

71

После приведения к нижнему регистру значений в столбце education было выявлено 71 дубликатов, несмотря на то, что нет каких-либо данных идентифицирующих самого клиента, данные о стаже работы и о доходе указывают на то, что клиенты дублируются, поэтому следует удалить дубликаты.

In [32]:
df = df.drop_duplicates().reset_index(drop=True)

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [33]:
education_log = df[['education', 'education_id']]
df = df.drop('education', 1)
family_status_log = df[['family_status', 'family_status_id']]
df = df.drop('family_status', 1)
display(education_log.head())
display(family_status_log.head())
df.head()

C:\Users\1\AppData\Local\Temp/ipykernel_10176/2421101579.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('education', 1)
C:\Users\1\AppData\Local\Temp/ipykernel_10176/2421101579.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('family_status', 1)


,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1


,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,1


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


В полученных словарях для определения education_id и family_status_id обнружено множество дубликатов от которых нужно избавиться.

In [34]:
education_log = education_log.drop_duplicates().reset_index(drop=True)
display(education_log)
family_status_log = family_status_log.drop_duplicates().reset_index(drop=True)
family_status_log

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


### Категоризация дохода.

In [35]:
# добавляю столбец с категорией дохода
def income_category(total_income):
    if 0 <= total_income <= 30000:
        return 'E'
    if 30001 <= total_income <= 50000:
        return 'D'
    if 50001 <= total_income <= 200000:
        return 'C'
    if 200001 <= total_income <= 1000000:
        return 'B'
    return 'A'
df['total_income_category'] = df['total_income'].apply(income_category)
df['total_income_category'].value_counts(dropna=False)

C    16016
B     5040
D      350
A       25
E       22
Name: total_income_category, dtype: int64

### Категоризация целей кредита.

In [36]:
# столбец с категориями целей кредита
def purpose_category(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'жил' in purpose or 'недвиж' in purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образов' in purpose:
        return 'получение образования'    
df['purpose_category'] = df['purpose'].apply(purpose_category)
df['purpose_category'].value_counts(dropna=False)

операции с недвижимостью    10810
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64

### Итоги предобработки данных
В результате предварительной обработки данных:
1. Были обнаружены 
    - пропуски, 
    - артефактные значения, 
    - неподходящие для исследования типы данных 
    - небольшое количество дубликатов
2. Датасет был представлен в "неудобно" читаемом виде
3. Были заполнены пропуски в 'total_income' и 'days_employed'
4. Исправлены аномальные данные в 'children', в 'total_income' 
5. Изменен тип данных
6. Удалены строки-дубликаты
7. Созданы отдельные словари для простого определения уровня образования и семейного положения
8. Доходы и цели кредита были категоризированы

## Исследовательский анализ.

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [37]:
children_debt_pivot = df.pivot_table(index='children', values='debt', aggfunc=['count', 'sum', 'mean'])
children_debt_pivot.columns = ['total_in_gr', 'debt_in_gr', 'debt_part']
children_debt_pivot

,total_in_gr,debt_in_gr,debt_part
children,,,
0,14090,1063,0.075444
1,4855,445,0.091658
2,2128,202,0.094925
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


#### Вывод :
Доля должников от общего количества клиентов, сгруппированных по количеству детей, по каждой группе не превышает 10%, из чего можем сделать вывод, что факт наличия или отсутствия детей не является критерием возврата кредита в срок. Вместе с тем следует обратить внимание, что в наиболее многочисленных группах, где у клиентов нет детей совсем или один-два ребенка, процент должников у бездетных ниже примерно на 2%, чем у групп с 1-2 детьми.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [38]:
family_status_id_pivot = df.pivot_table(index='family_status_id', values='debt', aggfunc=['count', 'sum', 'mean'])
family_status_id_pivot.columns = ['total_in_gr', 'debt_in_gr', 'debt_part']
family_status_id_pivot = family_status_id_pivot.merge(family_status_log, on='family_status_id', how='left')
family_status_id_pivot

,family_status_id,total_in_gr,debt_in_gr,debt_part,family_status
0,0,12339,931,0.075452,женат / замужем
1,1,4150,388,0.093494,гражданский брак
2,2,959,63,0.065693,вдовец / вдова
3,3,1195,85,0.071130,в разводе
4,4,2810,274,0.097509,Не женат / не замужем


#### Вывод:
Доля должников в зависимости от семейного положения не имеет значительных отличий, хотя доля должников среди тех, у кого не было задокументированного статуса (гражданский брак и не женатые) чуть больше, чем в группах, где семейное положение было оформлено документально, хоть разница не существенная, при принятии решения о выдаче кредита стоит учитывать факт семейного положения.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [39]:
income_category_pivot = df.pivot_table(index='total_income_category', values='debt', aggfunc=['count', 'sum', 'mean'])
income_category_pivot.columns = ['total_in_gr', 'debt_in_gr', 'debt_part']
income_category_pivot

,total_in_gr,debt_in_gr,debt_part
total_income_category,,,
A,25,2,0.080000
B,5040,356,0.070635
C,16016,1360,0.084915
D,350,21,0.060000
E,22,2,0.090909


#### Вывод:
В целом, соотношение должников по группам не имеет серьезных отличий и никак не привязано к снижению дохода. Группы 'А' и 'Е' малочисленные, если результаты по ним не брать в рассчет, то видим, что наиболее рисковая группа - клиенты с категорией дохода 'С' (от 50 000 до 200 000р), это наиболее многочисленная группа, чтобы связывать влияние уровня дохода с решением по кредиту, потребуется дополнительный анализ, где группу 'С' можно рассмотреть в разрезе городов, так как верхнее и нижнее значение по группе указывает на совершенно разное материальное положенеи в малых и крупных городах.

### Как разные цели кредита влияют на его возврат в срок?

In [40]:
purpose_category_pivot = df.pivot_table(index='purpose_category', values='debt', aggfunc=['count', 'sum', 'mean'])
purpose_category_pivot.columns = ['total_in_gr', 'debt_in_gr', 'debt_part']
purpose_category_pivot

,total_in_gr,debt_in_gr,debt_part
purpose_category,,,
операции с автомобилем,4306,403,0.093590
операции с недвижимостью,10810,782,0.072340
получение образования,4013,370,0.092200
проведение свадьбы,2324,186,0.080034


#### Вывод:
В зависимости от цели кредита также не обнаружено сущесственных отличий по группам. Доля должников среди тех, кто берет деньги на операции с автомобилем и получение образования, хоть и не сильно, но больше, то есть эта цель потенциально несет больше рисков, а наименее рисковой группой являются операции с недвижимостью.

## Общий вывод:

В ходе работы были заполнены пропуски данных, удалены дубликаты, добавлены ноые столбца, облегчающие анализ данных.
Рекомендуется проверить корректность сбора данных в days_employed, так как отрицательные и "нереалистичные" значения могут помешать при проверке новых гипотез.

Семейное положение и количество детей клиента не оказывает существенного влияния на факт погашения кредита в срок, тем не мение некоторые отличия по группам присутствуют, поэтому в скоринге следует учитывать, что если семейное положение было зарегистрированно официально, клиент более ответственно подходит к погашению кредита, а также отсутствие детей немного повышает возможность вернуть кредит в срок. Также стоит оценивать цель получения кредита, где операции с недвижимостью являются наименее рисковыми. Информация о категории доходов в представленном виде не дает возможности оценить риски, для более точного анализа потребуютя данные о городах проживания/работы. 